In [1]:
import pandas as pd
import requests
import os
import zipfile
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

diretorio_atual = os.getcwd()
os.chdir(f"{diretorio_atual}/dados_cvm")

In [ ]:
anos = range(2010, 2023)

url_base = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/"

for ano in anos:
    
    download = requests.get(url_base + f"dfp_cia_aberta_{ano}.zip")
    
    open(f"dfp_cia_aberta_{ano}.zip", "wb").write(download.content)

In [2]:
lista_demontracoes_2010_2022 = []

diretorio_atual = os.getcwd()

for arquivo in os.listdir(diretorio_atual):
    
    if arquivo == ".ipynb_checkpoints":
        
        pass
    
    elif arquivo == '.env':
        
        pass
    
    else:
        
        arquivo_zip = zipfile.ZipFile(arquivo)
        
        for planilha in arquivo_zip.namelist():
            
            demonstracao = pd.read_csv(arquivo_zip.open(planilha), sep = ";", encoding = 'ISO-8859-1',
                                      dtype = {"ORDEM_EXERC": "category"})
            
            lista_demontracoes_2010_2022.append(demonstracao)
            


In [3]:
base_dados = pd.concat(lista_demontracoes_2010_2022)

base_dados

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,CATEG_DOC,ID_DOC,DT_RECEB,LINK_DOC,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF
0,00.000.000/0001-91,2012-12-31,1,BCO BRASIL S.A.,1023,DFP,24372.0,2013-02-21,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00.000.000/0001-91,2012-12-31,2,BCO BRASIL S.A.,1023,DFP,24598.0,2013-03-01,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00.000.000/0001-91,2012-12-31,3,BCO BRASIL S.A.,1023,DFP,25253.0,2013-03-25,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00.000.000/0001-91,2012-12-31,4,BCO BRASIL S.A.,1023,DFP,25257.0,2013-03-25,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00.000.208/0001-00,2012-12-31,1,BRB BCO DE BRASILIA S.A.,14206,DFP,24834.0,2013-03-11,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61860,97.837.181/0001-47,2022-12-31,1,DEXCO S.A.,21091,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2022-12-31,7.08.04.02,Dividendos,0.0,S,2022-01-01,NaN
61861,97.837.181/0001-47,2022-12-31,1,DEXCO S.A.,21091,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2021-12-31,7.08.04.03,Lucros Retidos / Prejuízo do Período,847006.0,S,2021-01-01,NaN
61862,97.837.181/0001-47,2022-12-31,1,DEXCO S.A.,21091,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2022-12-31,7.08.04.03,Lucros Retidos / Prejuízo do Período,506861.0,S,2022-01-01,NaN
61863,97.837.181/0001-47,2022-12-31,1,DEXCO S.A.,21091,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2021-12-31,7.08.05,Outros,0.0,S,2021-01-01,NaN


In [ ]:
base_dados[['con_ind', 'tipo_dem']] = base_dados['GRUPO_DFP'].str.split("-", expand = True)
base_dados['con_ind'] = base_dados['con_ind'].str.strip()
base_dados['tipo_dem'] = base_dados['tipo_dem'].str.strip()

base_dados = base_dados[base_dados['ORDEM_EXERC'] != "PENÚLTIMO"]

In [ ]:
import time

In [ ]:
time.sleep(3)
url_cvm = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = url_cvm
driver.get(url)

In [ ]:
lista_dem = base_dados['tipo_dem'].unique()

In [ ]:
lista_empresas = base_dados['DENOM_CIA'].unique()

In [ ]:
weg_dre = base_dados[(base_dados["DENOM_CIA"] == "WEG S.A.") & 
                     (base_dados["tipo_dem"] == "Demonstração do Resultado") &
                    (base_dados["DS_CONTA"] == "Receita de Venda de Bens e/ou Serviços") &
                    (base_dados["con_ind"] == "DF Consolidado")]

weg_dre = weg_dre[['DT_REFER', 'VL_CONTA']]

weg_dre